# **50.007 ML 1D Project**
Failed Attempts , not actual p4

## Setup

In [76]:
import numpy as np
import os
import copy as copy
from collections import Counter

In [77]:
cwd = os.getcwd()
np.random.seed(1993)

## Reading Files

In [78]:
# Functions to read data

# Read dev.in data
def read_dev_in_data(filepath):
    results = []

    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            results.append(line.strip())
    
    return results

# Read dev.out data
def read_dev_out_data(filepath):
    results = []

    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            stripped_line = line.strip().split(" ")
            results.append(stripped_line)
    
    return results

# Read train data
def read_train_data(filepath):
    results = []

    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            stripped_line = line.strip().split(" ")
            results.append(stripped_line)

    return results

In [79]:
# Get pdath of the data
#------------------------------------
# Spanish: ES
ES_dev_in_data_path = os.path.join("Data", "ES" , "dev.in")
ES_dev_out_data_path = os.path.join("Data", "ES" , "dev.out")
ES_train_data_path = os.path.join("Data", "ES" , "train")

# Russiadn: RU
RU_dev_in_data_path = os.path.join("Data", "RU" , "dev.in")
RU_dev_out_data_path = os.path.join("Data", "RU" , "dev.out")
RU_train_data_path = os.path.join("Data", "RU" , "train")
#------------------------------------

## Part 1

In [80]:
# Split words and tags
def split_words_tags(labeled_data):
        words = []
        tags = []

        for word_tag in labeled_data:
            
            if len(word_tag) != 2:
                continue
            
            #word_tag is a list
            word = word_tag[0]
            tag = word_tag[1]

            words.append(word)
            tags.append(tag)

        return words, tags

# Count unique tags
def count_unique_tags(tags_ls):

    tags_unique = set()
    for tag in tags_ls:
        tags_unique.add(tag)
    return tags_unique

# Count unique words
def count_unique_words(words_ls):
         
    words_unique = set()
    for word in words_ls:
        words_unique.add(word)
    return words_unique

In [81]:
# Emission Parameters

# Get the emission parameters
def get_emission_parameters(ls_of_tags, ls_of_words, tags, words, k=1):

  # Write a function that estimates the emission parameters from the training set using MLE (maximumlikelihood estimation):
    # e(x|y) = Count(y -> x) / Count(y)
    # Count(y -> x) = Number of times word x is tagged with tag y
    # Count(y) = Number of times tag y appears

    # Input: ls_of_tags - list of unqiue tags
    # Input: ls_of_words - list of unqiue words
    # Input: tags - list of all tags
    # Input: words - list of all words
    # Output: emission_parameters

    # emission_parameters is a dictionary where:
        # The keys are (tag, word) tuples
        # The values are the emission parameters e(x|y)

    # Example of emission_parameters:
        # emission_parameters[("O", "apple")] = 0.00019841269
        # emission_parameters[("B-positive", "apple")] = 0.00000031622777

    # Create a dictionary to store the emission parameters
    emission_parameters = {}

    # Create a dictionary to store the count of each tag
    count_y = {}

    # Create a dictionary to store the count of each (tag, word) tuple
    count_y_to_x = {}

    # Get the count of each tag from the training set
    for tag_labels in ls_of_tags:
        count_y[tag_labels] = tags.count(tag_labels)
    
    print(f"This is Count(y) : {count_y}")

    # Get the count of each (tag, word) tuple from the training set
    for tag, word in zip(tags, words):
        if (tag, word) in count_y_to_x:
            count_y_to_x[(tag, word)] += 1
        else:
            count_y_to_x[(tag, word)] = 1

    print(f"This is Count(y -> x) : {count_y_to_x}")

    # Get the emission parameters
    for tag, word in count_y_to_x:

        emission_parameters[(tag, word)] = count_y_to_x[(tag, word)] / (count_y[tag] + k) # SOMETHING WRONG WITH THIS FORMULA
        if word == "con":
            print(tag,emission_parameters[(tag, word)])

    # For words that do not appear in the training set, k/(Count(y)+k) is used as the emission parameter
    unknown_word = "UNK"
    for tag in count_y:
        emission_parameters[(tag, unknown_word)] = k / (count_y[tag] + k)

    print(f"This is e(x|y) : {emission_parameters}")

    return emission_parameters

In [82]:
def assign_estimate_tags(test_words, emission_params, train_ls_of_words):

    # for each word in the test set of words (test_words) assign the tag with the highest emission probability

    # Inputs : test_tags - a list of all tags
    #          test_ls_of_tags - a list of unqiue tags
    #         test_number_of_tags - a list of the number of tags
    #        test_words - a list of all words
    #       emission_params - a dictionary of emission parameters
    # 
    # Output : labelled words - a list of words with their assigned tags

    predicted_results = []

    for word in test_words:
        if word in train_ls_of_words:

            # y∗ = arg max y e(x|y)
            emission_value = 0
            for key in emission_params:
                if key[1] == word:
                    if emission_value < emission_params[key]:
                        emission_value = emission_params[key]
                        value = key[0]
            
            predicted_results.append((word, value))
            
        else:

            if word != "":
                # y∗ = arg max y e(x|y)
                emission_value = 0
                for key in emission_params:
                    if key[1] == "UNK":
                        if emission_value < emission_params[key]:
                            emission_value = emission_params[key]
                            value = key[0]

                predicted_results.append(("UNK", value))
            
            else:
                predicted_results.append(("", ""))


    
    print("predicted_results: ", predicted_results)
    return predicted_results
    

In [83]:
def get_precision(test_labels, gold_standard):

    total_predicted = 0
    total_correct = 0

    # convert to set for faster lookup
    gold_standard_tuple_ver = []

    for tuple in gold_standard:

        if len(tuple) < 2:
            continue

        gold_standard_tuple_ver.append((tuple[0], tuple[1]))

    print(f"This is the gold standard: {gold_standard_tuple_ver} \n")

    for predicted_pair in test_labels:

        if predicted_pair in gold_standard_tuple_ver:
            total_correct += 1
    
        total_predicted += 1

    return total_correct / total_predicted

In [84]:
def get_recall(test_labels, gold_standard):

    total_correct = 0
    total_gold = 0

    gold_standard_tuple_ver = []

    for tuple in gold_standard:

        if len(tuple) < 2:
            continue

        gold_standard_tuple_ver.append((tuple[0], tuple[1]))

    for tuple in gold_standard_tuple_ver:

        if tuple in test_labels:
            total_correct += 1

        total_gold += 1

    return total_correct / total_gold

In [85]:
def get_f_score(precision, recall):
    return 2/((1/precision) + (1/recall))

In [86]:
def calculate_part_1(dev_in_data_path, dev_out_data_path, train_data_path, output_path):

    train_data = read_train_data(train_data_path)

    train_words, train_tags = split_words_tags(train_data)
    train_ls_of_tags = count_unique_tags(train_tags)
    train_ls_of_words = count_unique_words(train_words)

    # Get Emmission Parameters
    k = 1
    emission_params = get_emission_parameters(train_ls_of_tags, train_ls_of_words, train_tags, train_words, k)

    test_data = read_dev_in_data(dev_in_data_path)

    # Get labels for test data
    test_labels = assign_estimate_tags(test_data, emission_params, train_ls_of_words)

    with open(output_path, "w+", encoding="utf-8") as file:
        for line in test_labels:
            write_line = line[0] + " " + line[1] + "\n"
            file.write(write_line)

In [87]:
# Different Language

# For Spanish

print("For Spanish: ")
output_path = os.path.join("Data", "ES" , "dev.p1.out")
#uncomment to run
# calculate_part_1(ES_dev_in_data_path, ES_dev_out_data_path, ES_train_data_path, output_path)

print("\n")

# For Russian
print("For Russian: ")
output_path = os.path.join("Data", "RU" , "dev.p1.out")
#uncommen t to run
# calculate_part_1(RU_dev_in_data_path, RU_dev_out_data_path, RU_train_data_path, output_path)

For Spanish: 


For Russian: 


# Part IV

# Decoding with Forward-Backward

In [88]:
train_data = read_train_data(ES_train_data_path)
train_words, train_tags = split_words_tags(train_data)
train_ls_of_tags = count_unique_tags(train_tags)
train_ls_of_words = count_unique_words(train_words)
# Get Emmission Parameters
k = 1
emission_params = get_emission_parameters(train_ls_of_tags, train_ls_of_words, train_tags, train_words, k)

This is Count(y) : {'I-neutral': 43, 'B-positive': 1160, 'O': 29035, 'B-neutral': 72, 'I-negative': 171, 'B-negative': 381, 'I-positive': 314}
This is Count(y -> x) : {('O', 'Estuvimos'): 6, ('O', 'hace'): 26, ('O', 'poco'): 55, ('O', 'mi'): 72, ('O', 'pareja'): 13, ('O', 'y'): 1024, ('O', 'yo'): 36, ('O', 'comiendo'): 10, ('O', 'resultó'): 4, ('O', 'todo'): 115, ('O', 'muy'): 396, ('O', 'bien'): 165, ('O', ','): 1664, ('O', 'tanto'): 39, ('O', 'la'): 755, ('B-positive', 'comida'): 169, ('O', 'el'): 642, ('B-positive', 'vino'): 6, ('B-positive', 'trato'): 44, ('B-positive', 'decoración'): 7, ('O', '…'): 45, ('O', 'nos'): 146, ('O', 'gustó'): 11, ('O', 'mucho'): 53, ('O', '.'): 1623, ('O', 'Por'): 30, ('O', 'poner'): 5, ('O', 'algún'): 6, ('O', 'pero'): 191, ('O', 'quizá'): 4, ('B-negative', 'jamón'): 1, ('O', 'no'): 369, ('O', 'era'): 52, ('O', 'lo'): 270, ('O', '"'): 39, ('O', 'ibérico'): 2, ('O', 'que'): 845, ('O', 'cabía'): 1, ('O', 'esperar'): 5, ('O', 'Bien'): 6, ('O', 'sabe'): 5,

In [89]:
def read_train_data_p2(filepath):
    results = []
    #Add start state
    results.append(' START')

    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            stripped_line = line.strip()
            results.append(stripped_line)
            if stripped_line == "":
                # include stop and start states at new sentence
                results.append(' STOP')
                results.append(' START')
    final_results = []
    for line in results:
        split_lines = line.split(" ")
        final_results.append(split_lines)
    final_results.pop()
    #remove final start state

    return final_results

In [90]:
def get_transmission_parameters(ls_of_tags, tags):
    #Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation)
    # q( y_i | y_i-1 ) = Count( y_i-1, y_i ) / Count( y_i-1 )
    # Count(y_i-1 , y_i) = Number of times tag y_i-1 transits to tag y_i
    # Count(y_i-1) = Number of times tag y_i-1 appears

    # Input: ls_of_tags - list of unique tags
    # Input: tags - list of all tags

    # transmission_parameters is a dictionary where:
        # The keys are (tag_y_i-1, tag_y_i) tuples
        # The values are the transmission parameters q(y_i | y_i-1)

    # Example of emission_parameters:
        # emission_parameters[("O", "O")] = 0.00019841269
        # emission_parameters[("B-positive", "O")] = 0.00000031622777

    # Create a dictionary to store the emission parameters
    transmission_parameters = {}

    # Create a dictionary to store the count of each tag
    count_y = {}

    # Create a dictionary to store the count of each (y_i-1, y_i) tuple
    count_y_i_1_to_y_i = {}

    # Get the count of each tag from the training set
    for tag_labels in ls_of_tags:
        count_y[tag_labels] = tags.count(tag_labels)

    print(f"This is Count(y) : {count_y}")

    # Get the count of each (y_i-1, y_i) tuple from the training set
    for i in range(1, len(tags)):
        if (tags[i-1], tags[i]) in count_y_i_1_to_y_i:
            count_y_i_1_to_y_i[(tags[i-1],tags[i])] +=1
        else:
            count_y_i_1_to_y_i[(tags[i-1],tags[i])] =1

    print(f"This is Count (y_i-1 , y_i) : {count_y_i_1_to_y_i}")

    #transmission probability from state y_i-1 to y_i e.g ("START", "O") = 0.9281 == 0.9281 probability to transmit from "START" to "O" state

    for key, value in count_y_i_1_to_y_i.items():
        transmission_parameters[key] = value / count_y[key[0]]

    print(f"This is the q(y_i | y_i-1): {transmission_parameters}")

    labels = ["START", "STOP", "O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative"]
    for i in labels:
        for j in labels:
            if (i, j) in transmission_parameters:
                continue
            else:
                transmission_parameters[(i, j)] = 0

    return transmission_parameters

In [91]:
# note that if we just blindly shove the fractions into the algorithm
# multiply fractions enough times and it'll approach 0
# and yeah that's gonna end up becoming 0 ft. computer inaccuracy
# that's the numerical underflow
# we can prevent this by log-ing everything

# both transition_parameters and emission_parameters are dictionaries

def log_underflow_prevention(parameter_dict):
    log_parameter_dict = {}
    for key, value in parameter_dict.items():
        if value == 0:
            log_parameter_dict[key] = -np.inf
        else:
            log_parameter_dict[key] = np.log(value)
    return log_parameter_dict

def constant_underflow_prevention(parameter_dict):
    constant_parameter_dict = {}
    for key, value in parameter_dict.items():
        if value == 0:
            constant_parameter_dict[key] = 0
        else:
            constant_parameter_dict[key] = 100*value
    return constant_parameter_dict


In [92]:
# Reads the dev data
def read_dev(path):
  out = [[]]
  f = open(path, "r", encoding="utf-8")
  lines_in = f.readlines()
  for word in lines_in:
    if word == "\n":
      out.append([])
    else:
      out[-1].append(word.rstrip())
  return out[:-1]

In [93]:
def forward_backward_max_tags(document, transmission, emission, ls_of_words):
    n = len(document)
    tags = ["O", "B-positive", "B-neutral", "B-negative", "I-positive", "I-neutral", "I-negative", "STOP"]
    
    # Forward Algorithm
    # Corrected initialization of forward and backward dictionaries
    forward = [{u: 0.0 for u in tags} for _ in range(n + 1)]
    backward = [{u: 0.0 for u in tags} for _ in range(n + 1)]

    forward_final = [{u: 0.0 for u in tags} for _ in range(n + 1)]
    backward_final = [{u: 0.0 for u in tags} for _ in range(n + 1)]

    
    forward[0]["START"] = 100 # Initialize the START state

    # HMM IV forward probability eq 2
    for tag in tags:
        transmission_u = transmission.get(("START", tag)) or 0
        # base case
        forward_final[1][tag] = transmission_u
    
    # HMM IV forward probability eq 3, recursive
    for j in range(2, n + 1):
        for u in tags:
            forward[j][u] = 0
            for v in tags:
                if document[j - 1] in ls_of_words:
                    emission_prob = emission.get((u, document[j - 1])) or 0
                    # print(emission_prob)
                else:
                    emission_prob = emission.get((u, "UNK")) or 0
                transmission_v_u = transmission.get((v, u)) or 0
                probability_temp = forward[j][u]
                forward[j][u] = forward_final[j - 1][v] * transmission_v_u * emission_prob + probability_temp
                # if j == 2: #otherwise the console prints will clog up our VScode
                #     print( "1 =", forward_final[j-1][v], transmission_v_u, emission_prob)
                #     print("2 =", forward[j][u])
            forward_final[j][u] = forward[j][u]
    
    # HMM IV backward probability eq 5, base case
    for tag in tags:
        x_n = document[n - 1]
        if tag == "STOP":
            emission_prob = emission.get((tag, x_n)) or 0
            transmission_n = transmission.get((tag, "STOP")) or 0
            backward_final[n][tag] = transmission_n * emission_prob
    
    # HMM IV backward probability eq 6, recursive
    for j in range(n - 1, 0, -1):
        for u in tags:
            backward[j][u] = 0
            for v in tags:
                if document[j] in ls_of_words:
                    emission_prob = emission.get((v, document[j])) or 0
                else:
                    emission_prob = emission.get((v, "UNK")) or 0
                transmission_u_v = transmission.get((u, v)) or 0
                probability_temp = backward[j][u]
                backward[j][u] = backward_final[j + 1][v] * transmission_u_v * emission_prob + probability_temp
            backward_final[j][u] = backward[j][u]
    
    # Calculate Forward-Backward Probabilities and find the tag with maximum probability
    # aka eq 20
    max_tags = []
    for j in range(1, n + 1):
        max_prob = -np.inf
        max_tag = "O"
        for tag in tags:
            # HMM IV eq 20
            prob = forward_final[j][tag] * backward_final[j][tag]
            if prob > max_prob:
                max_prob = prob
                max_tag = tag
        max_tags.append(max_tag)
    
    return max_tags



In [94]:
def forward_backward_loop(data, transmission, emission, ls_of_words):
  results =[]
  for document in data:
    results.append(forward_backward_max_tags(document, transmission, emission, ls_of_words))
  return results

In [95]:
# Writes the prediction from trained data into the dev.in file and output
def assign_prediction(prediction, data, path):
    if (len(prediction) != len(data)):
        return "Error"
    file = open(path, "w", encoding="utf-8")
    n = len(data)
    for i in range(n):
        assert( len(prediction[i])== len(data[i]))
        m = len(data[i])
        for j in range(m):
            file.write(data[i][j] + " " + prediction[i][j] + "\n")
        file.write("\n")
    print("Wrote predictions to", path)
    return

In [96]:
def write_predictions_forward_backward(dev_in_data_path, dev_out_data_path, train_data_path, output_path):
  #sort train data into tag and words
  train_data = read_train_data(train_data_path)
  train_words, train_tags = split_words_tags(train_data)
  train_ls_of_tags = count_unique_tags(train_tags)
  train_ls_of_words = count_unique_words(train_words)
  # Get Emission Parameters
  k = 1
  emission_params = get_emission_parameters(train_ls_of_tags, train_ls_of_words, train_tags, train_words, k)

  #append start and stop to train data per document
  train_data_modified = read_train_data_p2(train_data_path)
  train_words_modified, train_tags_modified = split_words_tags(train_data_modified)
  train_ls_of_tags_modified = count_unique_tags(train_tags_modified)
  transmission_params = get_transmission_parameters(train_ls_of_tags_modified, train_tags_modified)

  #constant transmission and emission params to avoid underflow
  constant_emission = constant_underflow_prevention(emission_params)
  constant_transmission = constant_underflow_prevention(transmission_params)
  # read dev_in in list of list
  dev_in_list = read_dev(dev_in_data_path)
  
  # run forward_backward and get predictions for the whole dev_in
  predictions = forward_backward_loop(dev_in_list, constant_transmission, constant_emission, train_ls_of_words)
  assign_prediction(predictions, dev_in_list, output_path)
  return predictions

print("For Spanish: ")
output_path = os.path.join("Data", "ES" , "dev.forback.out")
print(write_predictions_forward_backward(ES_dev_in_data_path, ES_dev_out_data_path, ES_train_data_path, output_path))

# For Russian
print("For Russian: ")
output_path = os.path.join("Data", "RU" , "dev.forback.out")
print(write_predictions_forward_backward(RU_dev_in_data_path, RU_dev_out_data_path, RU_train_data_path, output_path))

For Spanish: 
This is Count(y) : {'I-neutral': 43, 'B-positive': 1160, 'O': 29035, 'B-neutral': 72, 'I-negative': 171, 'B-negative': 381, 'I-positive': 314}
This is Count(y -> x) : {('O', 'Estuvimos'): 6, ('O', 'hace'): 26, ('O', 'poco'): 55, ('O', 'mi'): 72, ('O', 'pareja'): 13, ('O', 'y'): 1024, ('O', 'yo'): 36, ('O', 'comiendo'): 10, ('O', 'resultó'): 4, ('O', 'todo'): 115, ('O', 'muy'): 396, ('O', 'bien'): 165, ('O', ','): 1664, ('O', 'tanto'): 39, ('O', 'la'): 755, ('B-positive', 'comida'): 169, ('O', 'el'): 642, ('B-positive', 'vino'): 6, ('B-positive', 'trato'): 44, ('B-positive', 'decoración'): 7, ('O', '…'): 45, ('O', 'nos'): 146, ('O', 'gustó'): 11, ('O', 'mucho'): 53, ('O', '.'): 1623, ('O', 'Por'): 30, ('O', 'poner'): 5, ('O', 'algún'): 6, ('O', 'pero'): 191, ('O', 'quizá'): 4, ('B-negative', 'jamón'): 1, ('O', 'no'): 369, ('O', 'era'): 52, ('O', 'lo'): 270, ('O', '"'): 39, ('O', 'ibérico'): 2, ('O', 'que'): 845, ('O', 'cabía'): 1, ('O', 'esperar'): 5, ('O', 'Bien'): 6, ('O

# Perceptron Implementation

Text Preprocessing (lowercase conversion) and Prior Knowledge of punctuation and numbers


In [97]:
# Writes the prediction from trained data into the dev.in file and output
def assign_prediction(prediction, data, path):
    if (len(prediction) != len(data)):
        return "Error"
    file = open(path, "w", encoding="utf-8")
    n = len(data)
    for i in range(n):
        assert( len(prediction[i])== len(data[i]))
        m = len(data[i])
        for j in range(m):
            file.write(data[i][j] + " " + prediction[i][j] + "\n")
        file.write("\n")
    print("Wrote predictions to", path)
    return

def write_predictions_part_2(dev_in_data_path, dev_out_data_path, train_data_path, output_path):
  #sort train data into tag and words
  train_data = read_train_data(train_data_path)
  train_words, train_tags = split_words_tags(train_data)
  train_ls_of_tags = count_unique_tags(train_tags)
  train_ls_of_words = count_unique_words(train_words)
  # Get Emission Parameters
  k = 1
  emission_params = get_emission_parameters(train_ls_of_tags, train_ls_of_words, train_tags, train_words, k)

  #append start and stop to train data per document
  train_data_modified = read_train_data_p2(train_data_path)
  train_words_modified, train_tags_modified = split_words_tags(train_data_modified)
  train_ls_of_tags_modified = count_unique_tags(train_tags_modified)
  transmission_params = get_transmission_parameters(train_ls_of_tags_modified, train_tags_modified)

  #log transmission and emission params to avoid underflow
  log_emission = log_underflow_prevention(emission_params)
  log_transmission = log_underflow_prevention(transmission_params)
  # read dev_in in list of list
  dev_in_list = read_dev(dev_in_data_path)
  
  # run viterbi and get predictions for the whole dev_in
  predictions = forward_backward_loop(dev_in_list, log_transmission, log_emission, train_ls_of_words)
  # write predictions into dev.p2.out
  assign_prediction(predictions, dev_in_list, output_path)

def calculate_part_2(dev_in_data_path, dev_out_data_path, train_data_path, output_path):
  write_predictions_part_2(dev_in_data_path, dev_out_data_path, train_data_path, output_path)




In [98]:
def viterbi(document, transmission, emission, emission_char, ls_of_words):
  n = len(document)
  tags = ["O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative","STOP"]

  memo = [{} for _ in range(n+1)]
  parent_arr = [{} for _ in range(n+1)]
  punctuations = '!"#$%&\'()*+, -./:;<=>?@[\]^_`{|}~0123456789'
  prior_knowledge = {char: "O" for char in punctuations}

  #initial step from start to first node
  for tag in tags:
    a_v_u = transmission.get(("START", tag)) 
    if document[0] in ls_of_words:
      # if tag emits word, get emission, else -inf
      b_u = emission.get((tag, document[0])) or -np.inf
    else:
      # #if word not in document
      # b_u = emission.get((tag, "UNK"))  or -np.inf
      b_u_char = []
      for char in document[0]:
        if char in emission_char:
          b_u_char.append(emission_char.get((tag, char)) or -np.inf)
        else:
          b_u_char.append(emission_char.get((tag, "UNK")) or -np.inf)
      
      # Find most occured tag for b_u_char
      b_u = Counter(b_u_char).most_common(1)[0][1]

    memo[0][tag] =  a_v_u + b_u
    parent_arr[0][tag] = None
  #recursive
  for j in range(1,n):
    for u in tags:
      max_prob = -np.inf
      max_v = None
      for v in tags:
        
        if document[j] in ls_of_words:
          emission_prob = emission.get((u, document[j])) or -np.inf
        # else:
        #   emission_prob = emission.get((u, "UNK")) or -np.inf
        else:
          # #if word not in document
          # b_u = emission.get((tag, "UNK"))  or -np.inf
          emission_prob_char = []
          for char in document[0]:
            if char in emission_char:
              emission_prob_char.append(emission_char.get((u, char)) or -np.inf)
          else:
            emission_prob_char.append(emission_char.get((u, "UNK")) or -np.inf)

          emission_prob = Counter(emission_prob_char).most_common(1)[0][1]
        
        #handling prior knowledge 
        for letter in document[j]:
          if letter in prior_knowledge:
            emission_prob = np.log(1)
            break
        

          
        transmission_v_u = transmission.get((v, u)) or -np.inf 
        prob = memo[j-1][v] + transmission_v_u+ emission_prob
        if prob > max_prob:
            max_prob = prob
            max_v = v
      memo[j][u] = max_prob
      parent_arr[j][u] = max_v
  # Termination step

  max_prob = -np.inf
  max_v = None
  for tag in tags:
    prob = memo[n-1][tag] + transmission.get((tag, "STOP"))
    if prob > max_prob:
        max_prob = prob
        max_v = tag

  if max_prob != -np.inf:
    memo[n]['STOP'] = max_prob
    parent_arr[n]['STOP'] = max_v
    
  most_likely_sequence = ["" for _ in range(n)]
  if max_v == None:
        max_v = "O"
  # Backtrack to find the most likely path
  for t in range(n , 0, -1):
    max_v = parent_arr[t].get(max_v)
    if max_v == None:
      max_v = "O"
    most_likely_sequence[t-1] = max_v

  return most_likely_sequence

In [99]:
# Different Language

# For Spanish

print("For Spanish: ")
output_path = os.path.join("Data", "ES" , "dev.p2.out")
#uncomment to run
# calculate_part_2(ES_dev_in_data_path, ES_dev_out_data_path, ES_train_data_path, output_path)

print("\n")

# For Russian
print("For Russian: ")
output_path = os.path.join("Data", "RU" , "dev.p2.out")
#uncomment to run
# calculate_part_2(RU_dev_in_data_path, RU_dev_out_data_path, RU_train_data_path, output_path)

For Spanish: 


For Russian: 


## Perceptron Algorithm based on recommended research paper in project PDF

In [100]:
def getTrainWords(train_data_path):
  train_data = read_train_data(train_data_path)
  words = []
  for word_tag in train_data:
      if len(word_tag) != 2:
          words.append(word_tag[0])
      else:
        #word_tag is a list
        word = word_tag[0]
        words.append(word)

  return words

def get_initial_transmission_parameters(ls_of_tags, tags):
    #Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation)
    # q( y_i | y_i-1 ) = Count( y_i-1, y_i ) / Count( y_i-1 )
    # Count(y_i-1 , y_i) = Number of times tag y_i-1 transits to tag y_i
    # Count(y_i-1) = Number of times tag y_i-1 appears

    # Input: ls_of_tags - list of unique tags
    # Input: tags - list of all tags

    # transmission_parameters is a dictionary where:
        # The keys are (tag_y_i-1, tag_y_i) tuples
        # The values are the transmission parameters q(y_i | y_i-1)

    # Example of emission_parameters:
        # emission_parameters[("O", "O")] = 0.00019841269
        # emission_parameters[("B-positive", "O")] = 0.00000031622777

    # Create a dictionary to store the emission parameters
    transmission_parameters = {}

    # Create a dictionary to store the count of each tag
    count_y = {}

    # Create a dictionary to store the count of each (y_i-1, y_i) tuple
    count_y_i_1_to_y_i = {}

    # Get the count of each tag from the training set
    for tag_labels in ls_of_tags:
        count_y[tag_labels] = tags.count(tag_labels)


    # Get the count of each (y_i-1, y_i) tuple from the training set
    for i in range(1, len(tags)):
        if (tags[i-1], tags[i]) in count_y_i_1_to_y_i:
            count_y_i_1_to_y_i[(tags[i-1],tags[i])] +=1
        else:
            count_y_i_1_to_y_i[(tags[i-1],tags[i])] =1


    #transmission probability from state y_i-1 to y_i e.g ("START", "O") = 0.9281 == 0.9281 probability to transmit from "START" to "O" state

    for key, value in count_y_i_1_to_y_i.items():
        transmission_parameters[key] = value / count_y[key[0]]


    labels = ["START", "STOP", "O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative"]
    for i in labels:
        for j in labels:
            if (i, j) in transmission_parameters:
                continue
            else:
                transmission_parameters[(i, j)] = 0

    return count_y_i_1_to_y_i, transmission_parameters

    # Emission Parameters

# Get the emission parameters
def get_initial_emission_parameters(ls_of_tags, ls_of_words, tags, words, k=1):

  # Write a function that estimates the emission parameters from the training set using MLE (maximumlikelihood estimation):
    # e(x|y) = Count(y -> x) / Count(y)
    # Count(y -> x) = Number of times word x is tagged with tag y
    # Count(y) = Number of times tag y appears

    # Input: ls_of_tags - list of unqiue tags
    # Input: ls_of_words - list of unqiue words
    # Input: tags - list of all tags
    # Input: words - list of all words
    # Output: emission_parameters

    # emission_parameters is a dictionary where:
        # The keys are (tag, word) tuples
        # The values are the emission parameters e(x|y)

    # Example of emission_parameters:
        # emission_parameters[("O", "apple")] = 0.00019841269
        # emission_parameters[("B-positive", "apple")] = 0.00000031622777

    # Create a dictionary to store the emission parameters
    emission_parameters = {}

    # Create a dictionary to store the count of each tag
    count_y = {}

    # Create a dictionary to store the count of each (tag, word) tuple
    count_y_to_x = {}

    # Get the count of each tag from the training set
    for tag_labels in ls_of_tags:
        count_y[tag_labels] = tags.count(tag_labels)
    

    # Get the count of each (tag, word) tuple from the training set
    for tag, word in zip(tags, words):
        if (tag, word) in count_y_to_x:
            count_y_to_x[(tag, word)] += 1
        else:
            count_y_to_x[(tag, word)] = 1


    # Get the emission parameters
    for tag, word in count_y_to_x:

        emission_parameters[(tag, word)] = count_y_to_x[(tag, word)] / (count_y[tag] + k) # SOMETHING WRONG WITH THIS FORMULA

    # For words that do not appear in the training set, k/(Count(y)+k) is used as the emission parameter
    unknown_word = "UNK"
    for tag in count_y:
        emission_parameters[(tag, unknown_word)] = k / (count_y[tag] + k)


    return count_y_to_x, emission_parameters
    
from collections import Counter 
import random


def viterbi(document, transmission, emission, emission_char, chars, ls_of_words):
  n = len(document)
  tags = ["O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative","STOP"]

  memo = [{} for _ in range(n+1)]
  parent_arr = [{} for _ in range(n+1)]
  #initial step from start to first node
  for tag in tags:
    a_v_u = transmission.get(("START", tag)) 
    if document[0] in ls_of_words:
      # if tag emits word, get emission, else -inf
      b_u = emission.get((tag, document[0])) or -np.inf
    else:
      # #if word not in document
      # b_u = emission.get((tag, "UNK"))  or -np.inf
      b_u_char = []
      for char in document[0]:
        if (char in chars):
          b_u_char.append(emission_char.get((tag, char)) or -np.inf)
        else:
          b_u_char.append(emission_char.get((tag, "UNK")) or -np.inf)
      
      # Method 1 : Find most occured tag for b_u_char
      #b_u = Counter(b_u_char).most_common(1)[0][1]

      # Method 2 : Use probability of each tag for b_u_char and randomly select one
      b_u = random.choices(b_u_char).pop()

    memo[0][tag] =  a_v_u + b_u
    parent_arr[0][tag] = None

  #recursive
  for j in range(1,n):
    for u in tags:
      max_prob = -np.inf
      max_v = None
      for v in tags:
        if document[j] in ls_of_words:
          emission_prob = emission.get((u, document[j])) or -np.inf
        # else:
        #   emission_prob = emission.get((u, "UNK")) or -np.inf
        else:
          # #if word not in document
          # b_u = emission.get((tag, "UNK"))  or -np.inf
          emission_prob_char = []
          for char in document[j]:
            if (char in chars):
              emission_prob_char.append(emission_char.get((u, char)) or -np.inf)
            else:
              emission_prob_char.append(emission_char.get((u, "UNK")) or -np.inf)

          # # Method 1 : Find most occured tag for emission_prob_char
          # emission_prob = Counter(emission_prob_char).most_common(1)[0][1]

           # Method 2 : Use probability of each tag for emission_prob_char and randomly select one
          emission_prob = random.choices(emission_prob_char).pop()
        
        transmission_v_u = transmission.get((v, u)) or -np.inf 
        prob = memo[j-1][v] + transmission_v_u+ emission_prob
        if prob > max_prob:
            max_prob = prob
            max_v = v
      memo[j][u] = max_prob
      parent_arr[j][u] = max_v
  
  # Termination step

  max_prob = -np.inf
  max_v = None
  for tag in tags:
    prob = memo[n-1][tag] + transmission.get((tag, "STOP"))
    if prob > max_prob:
        max_prob = prob
        max_v = tag

  if max_prob != -np.inf:
    memo[n]['STOP'] = max_prob
    parent_arr[n]['STOP'] = max_v
    
  most_likely_sequence = ["" for _ in range(n)]
  if max_v == None:
        max_v = "O"
  # Backtrack to find the most likely path
  for t in range(n , 0, -1):
    max_v = parent_arr[t].get(max_v)
    if max_v == None:
      max_v = "O"
    most_likely_sequence[t-1] = max_v

  return most_likely_sequence

def viterbi_loop(data, transmission, emission, emission_char, chars, ls_of_words):
  results =[]
  for document in data:
    results.append(viterbi(document, transmission, emission, emission_char, chars, ls_of_words))
  return results

def initialize_parameters(train_data_path):
  #sort train data into tag and words
  train_data = read_train_data(train_data_path)
  train_words, train_tags = split_words_tags(train_data)
  train_ls_of_tags = count_unique_tags(train_tags)
  train_ls_of_words = count_unique_words(train_words)
  # Get Emission Parameters
  k = 1
  count_y_to_x, emission_params = get_initial_emission_parameters(train_ls_of_tags, train_ls_of_words, train_tags, train_words, k)

  #append start and stop to train data per document
  train_data_modified = read_train_data_p2(train_data_path)
  train_words_modified, train_tags_modified = split_words_tags(train_data_modified)
  train_ls_of_tags_modified = count_unique_tags(train_tags_modified)
  count_y_i_1_to_y_i, transmission_params = get_initial_transmission_parameters(train_ls_of_tags_modified, train_tags_modified)

  #log transmission and emission params to avoid underflow
  log_emission = log_underflow_prevention(emission_params)
  log_transmission = log_underflow_prevention(transmission_params)
  return count_y_i_1_to_y_i, log_transmission, count_y_to_x, log_emission, train_ls_of_words

def getTrainList(train_data):
  document = []
  result = []
  for word in train_data:
    if len(word) == 0:
      result.append(document)
      document= []
    else:
      document.append(word)

  return result

def get_tags_list_of_list(data_path):
  data = read_train_data(data_path)
  tags = []
  for tag in data:
      if len(tag) > 1:
          tags.append(tag[1])
      else:
        space = tag[0]
        tags.append(space)

  document = []
  result = []
  for tag in tags:
    if len(tag) == 0:
      result.append(document)
      document= []
    else:
        document.append(tag)

  return result

def get_words_tags_list(data_path):
    data = read_train_data(data_path)
    result = []
    words = []
    for word in data:
        if len(word) > 1:
            words.append(word)
        else:
          result.append(words)
          words = []
    return result
  

def get_new_transmission(train_tags, predicted_tags, count_y_i_1_to_y_i):
    assert(len(predicted_tags) ==  len(train_tags))
    new_count = count_y_i_1_to_y_i.copy()

    for i in range(len(predicted_tags)):
        #continue to next list if predicted list = train list
        if predicted_tags[i] == train_tags[i]:
            continue
        
        else:
          for j in range(1, len(predicted_tags[i])):
              new_count[(predicted_tags[i][j-1], predicted_tags[i][j])] = new_count.get((predicted_tags[i][j-1], predicted_tags[i][j]), 0) - 1
              new_count[(train_tags[i][j-1], train_tags[i][j])] = new_count.get((train_tags[i][j-1], train_tags[i][j]), 0) + 1
    return new_count


def get_new_emission(train_data_path, pred_data_path, count_y_to_x):
    new_count = count_y_to_x.copy()
    train_data = get_words_tags_list(train_data_path)
    pred_data = get_words_tags_list(pred_data_path)
    assert(len(train_data) == len(pred_data))
    for i in range(len(train_data)):
      if (train_data[i] == pred_data[i]):
         continue
      else:
         for j in range(len(pred_data[i])):
            if (train_data[i][j] == pred_data[i][j]):
                continue
            else:
               new_count[(train_data[i][j][1], train_data[i][j][0])] = new_count.get((train_data[i][j][1], train_data[i][j][0]),0) + 1
               new_count[(pred_data[i][j][1], pred_data[i][j][0])] = new_count.get((pred_data[i][j][1], pred_data[i][j][0]), 0) - 1
    return new_count



def calculate_emission_transmission(new_transmission, new_emission, k):
    count_y = {}
    emission_parameters = {}
    transmission_parameters = {}
    for pair in new_transmission:
       count_y[pair[0]] = count_y.get(pair[0], 0) + new_transmission.get(pair)
   # Get the emission parameters
    for tag, word in new_emission:

        emission_parameters[(tag, word)] = new_emission[(tag, word)] / (count_y[tag] + k) 

    # For words that do not appear in the training set, k/(Count(y)+k) is used as the emission parameter
    unknown_word = "UNK"
    for tag in count_y:
        emission_parameters[(tag, unknown_word)] = k / (count_y[tag] + k)

    #calculate new transmission params
    for key, value in new_transmission.items():
        transmission_parameters[key] = value / count_y[key[0]]


    labels = ["START", "STOP", "O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative"]
    for i in labels:
        for j in labels:
            if (i, j) in transmission_parameters:
                continue
            else:
                transmission_parameters[(i, j)] = 0

    return emission_parameters, transmission_parameters

def viterbi(document, transmission, emission, ls_of_words):
  n = len(document)
  tags = ["O", "B-positive", "B-neutral", "B-negative", "I-positive","I-neutral","I-negative","STOP"]

  memo = [{} for _ in range(n+1)]
  parent_arr = [{} for _ in range(n+1)]
  #initial step from start to first node
  for tag in tags:
    a_v_u = transmission.get(("START", tag)) 
    if document[0] in ls_of_words:
      # if tag emits word, get emission, else -inf
      b_u = emission.get((tag, document[0])) or -np.inf
    else:
      #if word not in document
      b_u = emission.get((tag, "UNK"))  or -np.inf
    memo[0][tag] =  a_v_u + b_u
    parent_arr[0][tag] = None
  #recursive
  for j in range(1,n):
    for u in tags:
      max_prob = -np.inf
      max_v = None
      for v in tags:
        if document[j] in ls_of_words:
          emission_prob = emission.get((u, document[j])) or -np.inf
        else:
          emission_prob = emission.get((u, "UNK")) or -np.inf
        transmission_v_u = transmission.get((v, u)) or -np.inf 
        prob = memo[j-1][v] + transmission_v_u+ emission_prob
        if prob > max_prob:
            max_prob = prob
            max_v = v
      memo[j][u] = max_prob
      parent_arr[j][u] = max_v
  # Termination step

  max_prob = -np.inf
  max_v = None
  for tag in tags:
    prob = memo[n-1][tag] + transmission.get((tag, "STOP"))
    if prob > max_prob:
        max_prob = prob
        max_v = tag

  if max_prob != -np.inf:
    memo[n]['STOP'] = max_prob
    parent_arr[n]['STOP'] = max_v
    
  most_likely_sequence = ["" for _ in range(n)]
  if max_v == None:
        max_v = "O"
  # Backtrack to find the most likely path
  for t in range(n , 0, -1):
    max_v = parent_arr[t].get(max_v)
    if max_v == None:
      max_v = "O"
    most_likely_sequence[t-1] = max_v

  return most_likely_sequence

def viterbi_loop(data, transmission, emission, ls_of_words):
    results =[]
    for document in data:
      results.append(viterbi(document, transmission, emission, ls_of_words))
    return results


def calculate_part_4(train_data_path, pred_data_path, k , iters, dev_in_data_path, output_path):
    # initialize training data parameters
    count_y_i_1_to_y_i, log_transmission, count_y_to_x, log_emission, train_ls_of_words = initialize_parameters(train_data_path)

    train_words = getTrainWords(train_data_path)
    #get the list of list of words for train set
    dev_train_list = getTrainList(train_words)
    # print(count_y_i_1_to_y_i)
    for i in range(iters):
        # run viterbi to get new parameters
        z = viterbi_loop(dev_train_list, log_transmission, log_emission, train_ls_of_words)
        assign_prediction(z , dev_train_list, pred_data_path)

        # #calculate new parameters 
        # train_tags = get_tags_list_of_list(train_data_path)
        # pred_tags = get_tags_list_of_list(pred_data_path)


        # new_transmission_count = get_new_transmission(train_tags, pred_tags, count_y_i_1_to_y_i)
        # new_emission_count = get_new_emission(train_data_path, pred_data_path, count_y_to_x)
        # emission_p, transmission_p = calculate_emission_transmission(new_transmission_count, new_emission_count, k)
        # emission_parameters = emission_p
        # transmission_parameters = transmission_p
        count_y_i_1_to_y_i, log_transmission, count_y_to_x, log_emission, train_ls_of_words = initialize_parameters(pred_data_path)
        print(count_y_i_1_to_y_i)
        
 
    # read dev_in in list of list
    dev_in_list = read_dev(dev_in_data_path)

    # run viterbi and get predictions for the whole dev_in
    predictions = viterbi_loop(dev_in_list, log_transmission, log_emission, train_ls_of_words)
    # write predictions into dev.p2.out
    assign_prediction(predictions, dev_in_list, output_path)

train_data_path = os.path.join("Data", "ES", "train")
pred_data_path = os.path.join("Data", "ES" , "dev.train.out")
k = 1
iters = 1
dev_in_data_path = os.path.join("Data", "ES", "dev.in")
output_path = os.path.join("Data", "ES", "dev.p4.out")
#uncomment to run
# calculate_part_4(train_data_path, pred_data_path, k, iters, dev_in_data_path, output_path)